In [92]:
from pprint import pprint
import numpy as np
import re
import nltk
import h5py
import io
import json
import JsonUtil
import tensorflow as tf
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Merge, Dropout, RepeatVector, merge, Flatten
from keras.layers import recurrent, Input, Bidirectional, LSTM, Lambda, GRU
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras import optimizers
## from rnnlayer import Attention, SimpleAttention
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [101]:
## loading the glove dictionary into the memory for usage in 
## making the word embeddings
GloveMatrix = {}
dimensionIndex = '300'
validationContext = [] 
validationQuestion = [] 
validationQuestion_id = [] 
validationToken2CharIdx = []
validationContextOriginal = []
maxLenvalidationContext = 0
maxLenvalidationQuestion = 0
trainContext = [] 
trainQuestion = [] 
trainQuestion_id = [] 
trainAnswerBegin = [] 
trainAnswerEnd = [] 
trainAnswerText = [] 
maxLenTrainingContext = 0
maxLenTrainingQuestion = 0
vocabulary = {}
word_index = {}
contextMaxlen = 0
questionMax = 0

In [38]:
## function for loading the glove files into the memory it
## will be used for vectorizing the vocabulary that we will be
## creating
def getGloveDictionary(gloveFile):
    f = open(gloveFile,'r')
    for line in f:
        values = line.split()
        currWord = values[0]
        vectorizedNotation = values[1:]
        ## converting vectorized notation to array so that we 
        ## can directly use this as an arrya
        vector = np.asarray(vectorizedNotation, dtype='float32')
        GloveMatrix[currWord] = vector 
    f.close()

In [37]:
## tokenizing the line
## returning the tokenized line with the tokens
def tokenize(currLine):
    finalTokens = []
    tokens = re.split('(\W+)?', currLine)
    for currToken in tokens:
        if (currToken.strip()):
            finalTokens.append(currToken.strip())
    return finalTokens

In [36]:
def removeCommonData(data):
    data = data.replace("``", '" ')
    data = data.replace("''", '" ')
    data = data.lower()
    return data

In [102]:
def processTrainingQues(qas, context, contextTokenized):
    global maxLenTrainingQuestion
    for qa in qas:
        question = qa._question_
        question = removeCommonData(question)
        questionTokenized = tokenize(question)
        if len(questionTokenized) > maxLenTrainingQuestion:
            maxLenTrainingQuestion = len(questionTokenized)
        question_id = qa._id_
        answers = qa._answers_
        for answer in answers:
            answerText = answer._text_
            answerTokenized = tokenize(answerText.lower())
            # find indices of beginning/ending words of answer span among tokenized context
            contextToAnswerFirstWord = context[:answer._answer_start_ + len(answerTokenized[0])]
            answerBeginIndex = len(tokenize(contextToAnswerFirstWord.lower())) - 1
            answerEndIndex = answerBeginIndex + len(answerTokenized) - 1
            trainQuestion.append(questionTokenized)
            trainQuestion_id.append(str(question_id))
            trainAnswerBegin.append(answerBeginIndex)
            trainAnswerEnd.append(answerEndIndex)
            trainAnswerText.append(answerText)
            trainContext.append(contextTokenized)

In [103]:
def splitDatasets(f):
    global maxLenTrainingContext
    for data in f:
        paragraphs = data._paragraphs_
        for paragraph in paragraphs:
            context = paragraph._context_
            context = removeCommonData(context)
            contextTokenized = tokenize(context)
            contextLength = len(contextTokenized)
            if contextLength > maxLenTrainingContext:
                maxLenTrainingContext = contextLength
            qas = paragraph._qas_
            processTrainingQues(qas, context, contextTokenized);

In [104]:
def getTokens(line):
    #tokens1 = nltk.word_tokenize(line)
    tokens = tokenize(line)
    ## removing bad datasets from the tokens
    for i in range(len(tokens)):
        tokens[i] = tokens[i].replace("''", '"')
        tokens[i].replace("``", '"')
    idx = 0;
    token_idx = 0;
    tokensStartIndex = range(0, len(tokens))   
    '''Code for getting the respective tokens and there respective index'''
    while idx < len(line) and token_idx < len(tokens):
        word = tokens[token_idx]
        if line[idx:idx+len(word)] == word:
            tokensStartIndex[token_idx] = idx
            idx = idx + len(word)
            token_idx = token_idx + 1 
        else:
            idx = idx + 1
    return tokens, tokensStartIndex

In [105]:
def processValidationQuestions(qas, tokenIdx2CharIdx, context, contextTokenized):
    global maxLenvalidationQuestion
    for qa in qas:
        question = qa._question_
        question = removeCommonData(question);
        questionTokenized = tokenize(question)
        if len(questionTokenized) > maxLenvalidationQuestion:
            maxLenvalidationQuestion = len(questionTokenized)
        question_id = qa._id_
        answers = qa._answers_
        validationQuestion.append(questionTokenized)
        validationQuestion_id.append(str(question_id))
        validationToken2CharIdx.append(tokenIdx2CharIdx)
        validationContextOriginal.append(context)
        validationContext.append(contextTokenized)

In [106]:
def splitValDatasets(f): 
    global maxLenvalidationContext
    for data in f:
        paragraphs = data._paragraphs_
        for paragraph in paragraphs:
            context = paragraph._context_
            context = removeCommonData(context);
            contextTokenized, tokenIdx2CharIdx = getTokens(context)
            contextLength = len(contextTokenized)
            if contextLength > maxLenvalidationContext:
                maxLenvalidationContext = contextLength
            qas = paragraph._qas_
            processValidationQuestions(qas, tokenIdx2CharIdx, context, contextTokenized);

In [128]:
def vectorizeTrainingData(dictionaryWordIndex, maximumcontext, maximumquestion):
    X = []
    Xq = []
    YBegin = []
    YEnd = []
    for i in range(len(trainContext)):
        x = [word_index[w] for w in trainContext[i]]
        xq = [word_index[w] for w in trainQuestion[i]]
        # map the first and last words of answer span to one-hot representations
        y_Begin =  np.zeros(len(trainContext[i]))
        y_Begin[trainAnswerBegin[i]] = 1
        y_End = np.zeros(len(trainContext[i]))
        y_End[trainAnswerEnd[i]] = 1
        X.append(x)
        Xq.append(xq)
        YBegin.append(y_Begin)
        YEnd.append(y_End)
    return pad_sequences(X, maxlen=maximumcontext, padding='post'),pad_sequences(Xq, maxlen=maximumquestion, padding='post'),pad_sequences(YBegin, maxlen=maximumcontext, padding='post'),pad_sequences(YEnd, maxlen=maximumcontext, padding='post')

In [129]:
def vectorizeValData(dictionaryWordIndex, maximumcontext, maximumquestion):
    X = []
    Xq = []
    YBegin = []
    YEnd = []
    for i in range(len(validationContext)):
        x = [word_index[w] for w in validationContext[i]]
        xq = [word_index[w] for w in validationQuestion[i]]
        X.append(x)
        Xq.append(xq)
    return pad_sequences(X, maxlen=maximumcontext, padding='post'), pad_sequences(Xq, maxlen=maximumquestion, padding='post')

In [108]:
def loadData():
    trainData = JsonUtil.import_qas_data('/Users/sumitbhanwala/Desktop/QustionAnsweringSystem/dataset/train-v1.1.json')
    splitDatasets(trainData)
    valData = JsonUtil.import_qas_data('/Users/sumitbhanwala/Desktop/QustionAnsweringSystem/dataset/dev-v1.1.json')
    splitValDatasets(valData)

In [125]:
'''Change the below function correctly'''
def getVocab():
    vocabulary = {}
    global maxLenTContext
    global maxLenVContext
    for words in trainContext:
        for word in words:
            if word in vocabulary:
                continue
            else:
                vocabulary[word] = 1
    for words in trainQuestion:
        for word in words:
            if word in vocabulary:
                continue
            else:
                vocabulary[word] = 1
    for words in validationContext:
        for word in words:
            if word in vocabulary:
                continue
            else:
                vocabulary[word] = 1
    for words in validationQuestion:
        for word in words:
            if word not in vocabulary:
                vocabulary[word] = 1
    vocabulary = sorted(vocabulary.keys())  
#     vocab_size = len(vocabulary) + 1
    dictionaryWordIndex = {}
    for i in range(len(vocabulary)):
        dictionaryWordIndex[vocabulary[i]]=i+1
    maximumcontext = 0
    maximumquestion = 0
    if (maxLenTrainingContext > maxLenvalidationContext):
        maximumcontext = maxLenTrainingContext
    else:
        maximumcontext = maxLenvalidationContext    
    if(maxLenvalidationQuestion > maxLenTrainingQuestion):
        maximumquestion = maxLenvalidationQuestion
    else:
        maximumquestion = maxLenTrainingQuestion
    return vocabulary, dictionaryWordIndex, maximumcontext, maximumquestion

In [131]:
def getVectorizedData(vocabulary, dictionaryWordIndex, maximumcontext, maximumquestion):    
    tX, tXq, tYBegin, tYEnd = vectorizeData(dictionaryWordIndex, maximumcontext, maximumquestion)
    randindex = np.random.permutation(tX.shape[0])
    tX = tX[randindex, :]
    tXq = tXq[randindex, :]
    tYBegin = tYBegin[randindex, :]
    tYEnd = tYEnd[randindex, :]
    vX, vXq = vectorizeValData(dictionaryWordIndex, maximumcontext, maximumquestion)
    return tX, tXq, tYBegin, tYEnd ,vX, vXq

In [34]:
def createEmbeddingMatrix():
    nb_words = len(word_index)
    EMBEDDING_DIM = int(GloveDimOption)
    MAX_SEQUENCE_LENGTH = context_maxlen
    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    # load pre-trained word embeddings into an Embedding layer
    embedding_layer = Embedding(nb_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

Preparing embedding matrix.


In [10]:
def main():
    glovePath = '/Users/sumitbhanwala/Desktop/glove.6B/glove.6B.' + str(dimensionIndex) + 'd.txt';
    getGloveDictionary(glovePath);
    loadData()
    vocabulary, dictionaryWordIndex, maximumcontext, maximumquestion = getVocab()
    tX, tXq, tYBegin, tYEnd ,vX, vXq = getVectorizedData(vocabulary, dictionaryWordIndex, maximumcontext, maximumquestion)
    ###
    createEmbeddingMatrix()
    cinput = Input(shape=(context_maxlen,), dtype='int32', name='cinput')
    cembedding=embedding_layer(cinput)
    qinput = Input(shape=(question_maxlen,), dtype='int32', name='qinput')
    qembedding=embedding_layer(qinput)
    # BiDirectional LSTM Layer
    D = Bidirectional(LSTM(150, return_sequences=False))(cembedding)
    # print(D.shape)
    Q = Bidirectional(LSTM(150, return_sequences=False))(qembedding)
    # print(Q.shape)
    L = merge([D, Q], mode='concat')
    ## Code for the Attention Layer 
    # print(qembedding.shape)
    # print(qembedding.shape)
    # Q = Bidirectional(LSTM(100, return_sequences=True))(qembedding)
    # Q1 = Dropout(0.2)(Q)
    # print(Q.shape)
    # clstm1 = Attention(100, Q1, 200, return_sequences=True, name='clstm1')(cembedding)
    # qlstm2 = Attention(100, clstm1, 100, return_sequences=True, name='qlstm2')(Q1)
    # clstm2 = Attention(100, qlstm2, 100, return_sequences=True, name='clstm2')(clstm1)
    # qlstm4 = Bidirectional(LSTM(100, name='qlstm4'))(qlstm3)
    # h = merge([qlstm4, clstm3], mode='concat', name='merge1'
    answerPtrBegin_output = Dense(context_maxlen, activation='softmax')(L)
    Lmerge = merge([L, answerPtrBegin_output], mode='concat', name='merge2')
    answerPtrEnd_output = Dense(context_maxlen, activation='softmax')(Lmerge)
    model = Model(input=[cinput, qinput], output=[answerPtrBegin_output, answerPtrEnd_output])
    adam = optimizers.Adam()#, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer=adam, loss='categorical_crossentropy',
                  loss_weights=[.04, 0.04], metrics=['accuracy'])
    model.summary()
    filepath1="weights/weights-matchlstm-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
    checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=False, mode='auto')
    checkpoint2 = EarlyStopping(monitor='val_dense_1_loss', min_delta=0.01, patience=3, verbose=0, mode='auto')
    callbacks_list = [checkpoint1, checkpoint2] 
    model.fit([tX, tXq], [tYBegin, tYEnd], epochs=10, batch_size=128, shuffle=True, validation_split=0.25, \
              callbacks=callbacks_list)
    predictions = model.predict([vX, vXq], batch_size=128)
    ansBegin = np.zeros((predictions[0].shape[0],), dtype=np.int32)
    ansEnd = np.zeros((predictions[0].shape[0],),dtype=np.int32) 
    for i in range(predictions[0].shape[0]):
        ansBegin[i] = predictions[0][i, :].argmax()
        ansEnd[i] = predictions[1][i, :].argmax()
    print(ansBegin.min(), ansBegin.max(), ansEnd.min(), ansEnd.max())
    answers = {}
    for i in range(len(vQuestion_id)):
        #print i
        if ansBegin[i] >= len(vContext[i]):
            answers[vQuestion_id[i]] = ""
        elif ansEnd[i] >= len(vContext[i]):
            answers[vQuestion_id[i]] = vContextOriginal[i][vToken2CharIdx[i][ansBegin[i]]:]
        else:
            answers[vQuestion_id[i]] = vContextOriginal[i][vToken2CharIdx[i][ansBegin[i]]:vToken2CharIdx[i][ansEnd[i]]+len(vContext[i][ansEnd[i]])]
    with io.open('predictedanswer.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(answers, ensure_ascii=False))

In [124]:
getVocab()

({u'anyeint': 8050,
  u'nunnery': 55414,
  u'localizes': 46889,
  u'sowell': 72966,
  u'sonja': 72752,
  u'vani': 81822,
  u'circuitry': 17949,
  u'spiders': 73244,
  u'seifer': 69919,
  u'hanging': 36096,
  u'woody': 84784,
  u'% ($': 181,
  u'godwar': 34364,
  u'localized': 46888,
  u'schlegel': 69244,
  u'lenca': 45907,
  u'cycling': 22079,
  u'\u6cb3': 87465,
  u'willibrordus': 84413,
  u'crossbar': 21507,
  u'ioannidis': 41263,
  u'grenadiers': 35024,
  u'raouf': 64278,
  u'bratislava': 13880,
  u'lamido': 45067,
  u'consideredt': 20058,
  u'broward': 14326,
  u'barcella': 10964,
  u'lusthaus': 47542,
  u'wcfl': 83700,
  u'wooded': 84753,
  u'vibrational': 82392,
  u'wooden': 84754,
  u'wednesday': 83796,
  u'salination': 68381,
  u'woods': 84770,
  u'magadhas': 47975,
  u'shows': 71154,
  u'immunities': 39378,
  u'hayreddin': 36553,
  u'chiasso': 17347,
  u'thrace': 77758,
  u'0054': 1127,
  u'270': 2741,
  u'271': 2743,
  u'272': 2744,
  u'273': 2746,
  u'274': 2747,
  u'275': 2

In [111]:
print(len(validationQuestion))

10570
